In [3]:
pip install yfinance

     |████████████████████████████████| 6.3MB 25.1MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.60-py2.py3-none-any.whl size=23819 sha256=a9565b10b45ec3a21cd67efc27571c528019e53d1c55edddfe0963f64f1f574f
  Stored in directory: /root/.cache/pip/wheels/f0/be/a4/846f02c5985562250917b0ab7b33fff737c8e6e8cd5209aa3b
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [4]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '${:,.2f}'.format

In [4]:
pip install --no-cache-dir -I pystan==2.19.1.1

     |████████████████████████████████| 67.3MB 1.6MB/s 
     |████████████████████████████████| 15.7MB 36.1MB/s 
     |████████████████████████████████| 2.0MB 33.5MB/s 
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.21.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [34]:
#today = datetime.today().strftime('%Y-%m-%d')
#start_date = '2016-01-01'
#eth_df = yf.download('ETH-USD',start_date, today)
#eth_df.tail()

In [21]:
eth_df=pd.read_csv('E6.csv',index_col='Date',parse_dates=True)

In [22]:
eth_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2031 entries, 2015-12-15 to 2021-07-06
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2027 non-null   float64
 1   High       2027 non-null   float64
 2   Low        2027 non-null   float64
 3   Close      2027 non-null   float64
 4   Adj Close  2027 non-null   float64
 5   Volume     2027 non-null   float64
dtypes: float64(6)
memory usage: 111.1 KB


In [23]:
eth_df.isnull().sum()

Open         4
High         4
Low          4
Close        4
Adj Close    4
Volume       4
dtype: int64

In [24]:
eth_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [25]:
eth_df.reset_index(inplace=True)
eth_df.columns 

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [26]:
df = eth_df[["Date", "Open"]]
new_names = {
    "Date": "ds", 
    "Open": "y",
}
df.rename(columns=new_names, inplace=True)

In [27]:
df.tail()


,ds,y
2026,2021-07-02,"$2,109.89"
2027,2021-07-03,"$2,150.83"
2028,2021-07-04,"$2,226.55"
2029,2021-07-05,"$2,321.92"
2030,2021-07-06,"$2,197.92"


In [28]:
# plot the open price
x = df["ds"]
y = df["y"]
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y))
# Set title
fig.update_layout(
    title_text="Time series plot of Ethereum Open Price",
)
# full code in notebook

In [29]:
m = Prophet(
    seasonality_mode="multiplicative" 
)
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [30]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
2391,2022-07-02
2392,2022-07-03
2393,2022-07-04
2394,2022-07-05
2395,2022-07-06


In [31]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2391,2022-07-02,"$3,737.81","$2,616.40","$4,778.91"
2392,2022-07-03,"$3,710.63","$2,596.58","$4,741.96"
2393,2022-07-04,"$3,719.74","$2,553.05","$4,782.37"
2394,2022-07-05,"$3,746.91","$2,603.29","$4,794.39"
2395,2022-07-06,"$3,752.24","$2,596.70","$4,818.15"


In [32]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()

2014.9173612960753

In [33]:
plot_plotly(m, forecast)

In [34]:
plot_components_plotly(m, forecast)